<a href="https://colab.research.google.com/github/Senhoril/Transfer_learn/blob/main/transfer_learn_dogs_cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

import os

#if using Theano with GPU
#os.environ["KERAS_BACKEND"] = "tensorflow"

import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import requests
import zipfile
import shutil
import glob
from keras.layers import Input

In [ ]:
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary()

553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ predictions (Dense)             │ (None, 1000)           │     4,097,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 138,357,544 (527.79 MB)

 Trainable params: 138,357,544 (527.79 MB)

 Non-trainable params: 0 (0.00 B)

# Task
Treinar um modelo de classificação de imagens para distinguir entre gatos e cachorros usando o dataset disponível em "https://download.microsoft.com/download/3/e/1/3e1c3f21-ecdb-4869-8368-6deba77b919f/kagglecatsanddogs_5340.zip", onde as imagens de cachorros estão na pasta "kagglecatsanddogs_5340/PetImages/Dog" e as imagens de gatos estão na pasta "kagglecatsanddogs_5340/PetImages/Cat".

## Baixar e extrair o dataset

### Subtask:
Baixar o arquivo zip do dataset e extrair as imagens para uma pasta local.


**Reasoning**:
Import necessary libraries and define variables for downloading and extracting the dataset.



In [ ]:
dataset_url = "https://download.microsoft.com/download/3/e/1/3e1c3f21-ecdb-4869-8368-6deba77b919f/kagglecatsanddogs_5340.zip"
zip_filename = "kagglecatsanddogs_5340.zip"
extraction_dir = "dataset_cats_dogs"

**Reasoning**:
Download the dataset zip file from the provided URL and save it locally.



In [ ]:
!echo "Downloading Kaggle Cats and Dogs zip file"
!curl -L -o kagglecatsanddogs.zip --progress-bar https://download.microsoft.com/download/3/e/1/3e1c3f21-ecdb-4869-8368-6deba77b919f/kagglecatsanddogs_5340.zip
!echo "Extracting the zip file"
!unzip -q kagglecatsanddogs.zip -d kagglecatsanddogs
!echo "Removing downloaded archive"
!rm kagglecatsanddogs.zip
!ls kagglecatsanddogs/PetImages

######################################################################## 100.0%
Extracting the zip file
Removing downloaded archive
Cat  Dog


In [ ]:
import os
import random
import shutil
import glob

# Define the root directory of the extracted dataset
original_dataset_dir = 'kagglecatsanddogs/PetImages'

# Define the base directory for the new organized dataset
base_dir = 'cats_and_dogs_filtered'

# Create the base directory
os.makedirs(base_dir, exist_ok=True)

# Define directories for training, validation, and test sets
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Remove existing directories if they exist to start fresh
if os.path.exists(train_dir):
    shutil.rmtree(train_dir)
if os.path.exists(validation_dir):
    shutil.rmtree(validation_dir)
if os.path.exists(test_dir):
    shutil.rmtree(test_dir)

os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Define directories for cat and dog images in each set
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

os.makedirs(train_cats_dir, exist_ok=True)
os.makedirs(train_dogs_dir, exist_ok=True)
os.makedirs(validation_cats_dir, exist_ok=True)
os.makedirs(validation_dogs_dir, exist_ok=True)
os.makedirs(test_cats_dir, exist_ok=True)
os.makedirs(test_dogs_dir, exist_ok=True)

# Get the list of image filenames
cat_fnames = [os.path.join(original_dataset_dir, 'Cat', fname) for fname in os.listdir(os.path.join(original_dataset_dir, 'Cat')) if fname.endswith('.jpg')]
dog_fnames = [os.path.join(original_dataset_dir, 'Dog', fname) for fname in os.listdir(os.path.join(original_dataset_dir, 'Dog')) if fname.endswith('.jpg')]

# Remove corrupted images and update the filenames list
cleaned_cat_fnames = []
for fpath in cat_fnames:
    try:
        img = plt.imread(fpath)
        if img is None:
            print(f"Corrupted image: {fpath}")
            # os.remove(fpath) # No need to remove from original data
        else:
            cleaned_cat_fnames.append(fpath)
    except:
        print(f"Corrupted image: {fpath}")
        # os.remove(fpath) # No need to remove from original data

cleaned_dog_fnames = []
for fpath in dog_fnames:
    try:
        img = plt.imread(fpath)
        if img is None:
            print(f"Corrupted image: {fpath}")
            # os.remove(fpath) # No need to remove from original data
        else:
            cleaned_dog_fnames.append(fpath)
    except:
        print(f"Corrupted image: {fpath}")
        # os.remove(fpath) # No need to remove from original data


cat_fnames = cleaned_cat_fnames
dog_fnames = cleaned_dog_fnames


# Shuffle the filenames
random.shuffle(cat_fnames)
random.shuffle(dog_fnames)

# Define split sizes with a smaller number of images
train_size = 50
validation_size = 25
test_size = 25

# Copy images to their respective directories
for fname in cat_fnames[:train_size]:
    shutil.copy(fname, train_cats_dir)
for fname in cat_fnames[train_size:train_size + validation_size]:
    shutil.copy(fname, validation_cats_dir)
for fname in cat_fnames[train_size + validation_size:train_size + validation_size + test_size]:
    shutil.copy(fname, test_cats_dir)

for fname in dog_fnames[:train_size]:
    shutil.copy(fname, train_dogs_dir)
for fname in dog_fnames[train_size:train_size + validation_size]:
    shutil.copy(fname, validation_dogs_dir)
for fname in dog_fnames[train_size + validation_size:train_size + validation_size + test_size]:
    shutil.copy(fname, test_dogs_dir)


print(f"Dataset organized with {train_size} images per class in train, {validation_size} in validation, and {test_size} in test sets.")

/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Dataset organized with 50 images per class in train, 25 in validation, and 25 in test sets.


In [ ]:
# Use VGG16 as the base model, excluding the top classification layers
vgg16_base = keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the VGG16 base model
for layer in vgg16_base.layers:
    layer.trainable = False

# Build the model on top of the VGG16 base
x = Flatten()(vgg16_base.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x) # 1 output node for binary classification

model = Model(inputs=vgg16_base.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,137,729 (80.63 MB)

 Trainable params: 6,423,041 (24.50 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

# Note that the validation data should not be augmented, only rescaled
validation_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir, # This is the target directory
        target_size=(224, 224), # All images will be resized to 224x224
        batch_size=32,
        class_mode='binary') # Since we use binary_crossentropy loss, we need binary labels

# Flow validation images in batches of 32 using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

Found 100 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [ ]:
# Train the model
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples // train_generator.batch_size,
      epochs=10, # You can adjust the number of epochs
      validation_data=validation_generator,
      validation_steps=validation_generator.samples // validation_generator.batch_size)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 59s 27s/step - accuracy: 0.4906 - loss: 6.5521 - val_accuracy: 0.5312 - val_loss: 2.7456
Epoch 2/10
1/3 ━━━━━━━━━━━━━━━━━━━━ 32s 16s/step - accuracy: 0.3750 - loss: 3.5678

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


3/3 ━━━━━━━━━━━━━━━━━━━━ 33s 8s/step - accuracy: 0.3750 - loss: 3.5678 - val_accuracy: 0.5312 - val_loss: 2.4057
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 105s 18s/step - accuracy: 0.4995 - loss: 2.1067 - val_accuracy: 0.5000 - val_loss: 1.7120
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 41s 12s/step - accuracy: 0.5000 - loss: 2.0470 - val_accuracy: 0.5312 - val_loss: 1.3106
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 143s 38s/step - accuracy: 0.5924 - loss: 1.4250 - val_accuracy: 0.5000 - val_loss: 1.1022
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 20s 9s/step - accuracy: 0.7500 - loss: 1.7499 - val_accuracy: 0.6250 - val_loss: 0.8791
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 143s 49s/step - accuracy: 0.6523 - loss: 1.0525 - val_accuracy: 0.6875 - val_loss: 0.6587
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 20s 9s/step - accuracy: 0.2500 - loss: 1.5765 - val_accuracy: 0.5625 - val_loss: 0.9820
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 82s 32s/step - accuracy: 0.5377 - loss: 1.2832 - val_accuracy: 0.6250 - val_loss: 0.6184
Epoch 10/10

In [ ]:
# Define data generator for the test set
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow test images in batches of 32 using test_datagen generator
test_generator = test_datagen.flow_from_directory(
        test_dir, # This is the target directory for test images
        target_size=(224, 224), # All images will be resized to 224x224
        batch_size=32,
        class_mode='binary',
        shuffle=False) # Do not shuffle test data for consistent evaluation

Found 50 images belonging to 2 classes.


In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)

print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step - accuracy: 0.8750 - loss: 0.2833
Test Loss: 0.2832685112953186
Test Accuracy: 0.875


In [ ]:
from keras.preprocessing import image
import numpy as np

# Function to load and preprocess an image
def load_and_preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) # Add batch dimension
    img_array /= 255.0 # Rescale to [0, 1]
    return img_array

# Load and preprocess the uploaded images
cat1_img = load_and_preprocess_image('/content/sample_data/cat1.jpg')
cat2_img = load_and_preprocess_image('/content/sample_data/cat2.jpg')

print("Images loaded and preprocessed.")

Images loaded and preprocessed.


In [ ]:
# Make predictions
cat1_prediction = model.predict(cat1_img)
cat2_prediction = model.predict(cat2_img)

# Interpret the predictions with confidence
# Assuming 'cat' is class 0 and 'dog' is class 1 based on alphabetical order
print(f"Prediction for cat1.jpg: {cat1_prediction[0][0]:.4f}")
if cat1_prediction[0][0] < 0.5:
    confidence = (1 - cat1_prediction[0][0]) * 100
    print(f"Predicted: Cat with {confidence:.2f}% confidence")
else:
    confidence = cat1_prediction[0][0] * 100
    print(f"Predicted: Dog with {confidence:.2f}% confidence")

print(f"Prediction for cat2.jpg: {cat2_prediction[0][0]:.4f}")
if cat2_prediction[0][0] < 0.5:
    confidence = (1 - cat2_prediction[0][0]) * 100
    print(f"Predicted: Cat with {confidence:.2f}% confidence")
else:
    confidence = cat2_prediction[0][0] * 100
    print(f"Predicted: Dog with {confidence:.2f}% confidence")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 494ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 539ms/step
Prediction for cat1.jpg: 0.0076
Predicted: Cat with 99.24% confidence
Prediction for cat2.jpg: 0.0900
Predicted: Cat with 91.00% confidence
